In [55]:
import pandas as pd

df = pd.read_csv('eval/llama/eval_antonio.csv')
def convert_to_float(val):
    # 1. Se c'è la E o e, è notazione scientifica: sostituisci la virgola con il punto
    if 'E' in val or 'e' in val:
        return float(val.replace(',', '.'))
    else:
        # 2. Altrimenti è un intero "umano": togli i punti
        return float(val.replace('.', ''))

df['ndcg_emb_phi'] = df['ndcg_emb_phi'].apply(convert_to_float)
df['ndcg_emb_phi'] = (df['ndcg_emb_phi'] / 1e16).round(6)
df

,id,modelId,layer,index,authorId,description,embedding,typeName,explanationModelName,umap_x,umap_y,umap_cluster,umap_log_feature_sparsity,true_description,ndcg_emb_phi,phi_decile,url,Soundness,⁠Completeness,Confidence
0,smuqtj03gin1a6ru2o9ujz5as,llama3.1-8b,0,7223,clkht01d40000jv08hvalcvly,['references to comfort and softness in descri...,NaN,['oai_token-act-pair'],['gpt-4o-mini'],0.0,0.0,0.0,0.0,references to comfort and softness in descript...,0.005397,5,https://neuronpedia.org/llama3.1-8b/0-llamasco...,3: Rilevante,1: Incompleto,4: certain
1,p6t6vi763nhuek9psdd9a32aj,llama3.1-8b,0,29280,clkht01d40000jv08hvalcvly,['references to the Associated Press and photo...,NaN,['oai_token-act-pair'],['gpt-4o-mini'],0.0,0.0,0.0,0.0,references to the Associated Press and photogr...,0.005025,4,https://neuronpedia.org/llama3.1-8b/0-llamasco...,2: Marginale,2: Parzialmente completo,4: certain
2,qbbr79uggoahcp705djxuhb10,llama3.1-8b,0,18918,clkht01d40000jv08hvalcvly,[' terms related to workspaces in a computatio...,NaN,['oai_token-act-pair'],['gpt-4o-mini'],0.0,0.0,0.0,0.0,terms related to workspaces in a computational...,0.002850,4,https://neuronpedia.org/llama3.1-8b/0-llamasco...,3: Rilevante,3: Completo,3: quite sure
3,yehvcswpdjvxf44awd1dtzu9x,llama3.1-8b,0,21175,clkht01d40000jv08hvalcvly,[' names of football players or related entiti...,NaN,['oai_token-act-pair'],['gpt-4o-mini'],0.0,0.0,0.0,0.0,names of football players or related entities,0.008177,5,https://neuronpedia.org/llama3.1-8b/0-llamasco...,1: Irrilevante,1: Incompleto,4: certain
4,yj0cw47s9yuvxekbx4a4f26gc,llama3.1-8b,0,29495,clkht01d40000jv08hvalcvly,"[' instances of the word ""mock"" or its variati...",NaN,['oai_token-act-pair'],['gpt-4o-mini'],0.0,0.0,0.0,0.0,"instances of the word ""mock"" or its variation...",0.004576,4,https://neuronpedia.org/llama3.1-8b/0-llamasco...,4: Molto rilevante,4: Molto completo,4: certain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,d1gtu4h4p7y5r9s0q3u48j7yv,llama3.1-8b,31,29361,clkht01d40000jv08hvalcvly,[' numerical identifiers and codes related to ...,NaN,['oai_token-act-pair'],['gpt-4o-mini'],0.0,0.0,0.0,0.0,numerical identifiers and codes related to art...,0.111706,7,https://neuronpedia.org/llama3.1-8b/31-llamasc...,4: Molto rilevante,2: Parzialmente completo,4: certain
96,shar0uwtzvt2unt8uavcoqbwb,llama3.1-8b,31,29156,clkht01d40000jv08hvalcvly,[' names and titles related to people and cult...,NaN,['oai_token-act-pair'],['gpt-4o-mini'],0.0,0.0,0.0,0.0,names and titles related to people and cultura...,0.027638,3,https://neuronpedia.org/llama3.1-8b/31-llamasc...,1: Irrilevante,1: Incompleto,4: certain
97,sk9b5i97op5nonb9qi18yg7n3,llama3.1-8b,31,29875,clkht01d40000jv08hvalcvly,[' references to musical performances and rela...,NaN,['oai_token-act-pair'],['gpt-4o-mini'],0.0,0.0,0.0,0.0,references to musical performances and related...,0.000020,0,https://neuronpedia.org/llama3.1-8b/31-llamasc...,1: Irrilevante,1: Incompleto,4: certain
98,jdjhhu2q1vt1j9p7qdagekhpp,llama3.1-8b,31,29342,clkht01d40000jv08hvalcvly,[' technical terms related to telecommunicatio...,NaN,['oai_token-act-pair'],['gpt-4o-mini'],0.0,0.0,0.0,0.0,technical terms related to telecommunications ...,0.048112,5,https://neuronpedia.org/llama3.1-8b/31-llamasc...,3: Rilevante,3: Completo,4: certain


In [60]:
# score computation for all the data ID

modello = 'gemini-2.0-flash'
for data_id in df['id'].unique():
    res = requests.post(
        "https://www.neuronpedia.org/api/explanation/score",
        headers={
            "Content-Type": "application/json",
            "X-Api-Key": "sk-np-xmcnRTVI05ZpA8Sg0y13jSDSM5LoZc0VPCVBdZzxCW00"
        },
        json={
            "explanationId": data_id,
            "scorerModel": modello,
            "scorerType": "eleuther_fuzz"
        }
    )
    
    response_json = json.loads(res.content)
    #add the score to the dataframe
    df.loc[df['id'] == data_id, f'score_{modello}'] = response_json['score']['value']
    df.loc[df['id'] == data_id, f'id_expl_{modello}'] = response_json['score']['explanationId']
    #print(f"Data ID: {data_id}, Score: {response_json['score']['value']}")

In [63]:
df.columns

Index(['id', 'modelId', 'layer', 'index', 'authorId', 'description',
       'embedding', 'typeName', 'explanationModelName', 'umap_x', 'umap_y',
       'umap_cluster', 'umap_log_feature_sparsity', 'true_description',
       'ndcg_emb_phi', 'phi_decile', 'url', 'Soundness', '⁠Completeness',
       'Confidence', 'score_gemini-2.0-flash', 'id_expl_gemini-2.0-flash'],
      dtype='object')

In [64]:
df1 = df[['id', 'modelId','true_description', 'layer', 'index','ndcg_emb_phi', 'score_gemini-2.0-flash', 'id_expl_gemini-2.0-flash']]

In [65]:
df1

,id,modelId,true_description,layer,index,ndcg_emb_phi,score_gemini-2.0-flash,id_expl_gemini-2.0-flash
0,smuqtj03gin1a6ru2o9ujz5as,llama3.1-8b,references to comfort and softness in descript...,0,7223,0.005397,0.950000,smuqtj03gin1a6ru2o9ujz5as
1,p6t6vi763nhuek9psdd9a32aj,llama3.1-8b,references to the Associated Press and photogr...,0,29280,0.005025,1.000000,p6t6vi763nhuek9psdd9a32aj
2,qbbr79uggoahcp705djxuhb10,llama3.1-8b,terms related to workspaces in a computational...,0,18918,0.002850,0.970588,qbbr79uggoahcp705djxuhb10
3,yehvcswpdjvxf44awd1dtzu9x,llama3.1-8b,names of football players or related entities,0,21175,0.008177,0.333333,yehvcswpdjvxf44awd1dtzu9x
4,yj0cw47s9yuvxekbx4a4f26gc,llama3.1-8b,"instances of the word ""mock"" or its variation...",0,29495,0.004576,1.000000,yj0cw47s9yuvxekbx4a4f26gc
...,...,...,...,...,...,...,...,...
95,d1gtu4h4p7y5r9s0q3u48j7yv,llama3.1-8b,numerical identifiers and codes related to art...,31,29361,0.111706,0.704545,d1gtu4h4p7y5r9s0q3u48j7yv
96,shar0uwtzvt2unt8uavcoqbwb,llama3.1-8b,names and titles related to people and cultura...,31,29156,0.027638,0.812500,shar0uwtzvt2unt8uavcoqbwb
97,sk9b5i97op5nonb9qi18yg7n3,llama3.1-8b,references to musical performances and related...,31,29875,0.000020,0.625000,sk9b5i97op5nonb9qi18yg7n3
98,jdjhhu2q1vt1j9p7qdagekhpp,llama3.1-8b,technical terms related to telecommunications ...,31,29342,0.048112,0.857143,jdjhhu2q1vt1j9p7qdagekhpp


In [66]:
df1.to_csv('eval/llama/score_llama.csv', index=False)